
# Implementing a Segmentation Model

## 1. Preparations
### 1.1 Install required packages

In [6]:
import json
import os
import geopandas as gpd
import sklearn
import torch
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'sklearn'

In [4]:
# !pip install segmentation-models-pytorch

In [5]:
import segmentation_models_pytorch as smp

ModuleNotFoundError: No module named 'torch'

### 1.2 Upload data to colab

In [ ]:
# Preprocessed data directories  # -------------------->> ADJUSTABLE
# GeoTIFFs
output_gtif_dir = r"D:\MoSE\preprocessing\gtif_outputs"
# NumpyArrays
output_nparr_dir = r"D:\MoSE\preprocessing\nparr_outputs"
# JSON class codes
output_json_dir = r"D:\MoSE\preprocessing\json_outputs"
# Shapefile class labels
shp_dir = r"G:\Meine Ablage\Dokumente.GD\FS06 SS24\BACHELORARBEIT\MoSE\data\shapefiles"


In [ ]:
# Class labels

#  Specific Shapefile path
shp_path = shp_dir + r"\GSK_24_WGS84_adjusted.shp"  # -------------------->> ADJUSTABLE

labels = gpd.read_file(shp_path) # read shapefile
labels_filtered = labels[labels["Elementtyp"].notnull()] # remove NULL
print(labels_filtered["Elementtyp"].unique()) # print all label classes

In [ ]:
# Class codes 

# path
label_codes_path = os.path.join(output_json_dir, "label_codes.json")   # -------------------->> ADJUSTABLE

# Open and load the JSON file
with open(label_codes_path, "r") as json_file:
    label_codes = json.load(json_file)

print(label_codes)
len(label_codes)

{'Einzelstein': 1, 'Wurzelstock': 2, 'Steinverbauung': 3, 'Totholz': 4, 'Steinriegel': 5, 'Schotterbank': 6, 'Schlamm_Sandinsel': 7, 'Sand_Schlammbank': 8, 'Schotterinsel': 9}


9

## 2. Choose segmentation model

In [ ]:
model = smp.Unet(   # -------------------->> ADJUSTABLE
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (3 for RGB)
    classes=len(label_codes),       # model output channels (number of classes)
)

NameError: name 'smp' is not defined

## 3. Configure data preprocessing

Datenvorverarbeitungsfunktion (preprocess_input) für ein bestimmtes Modell (Encoder) konfigurieren:

`from segmentation_models_pytorch.encoders import get_preprocessing_fn`

`preprocess_input = get_preprocessing_fn('resnet18', pretrained='imagenet')`

Encoder = der Teil des neuronalen Netzes, der Features aus den Eingabedaten extrahiert (in diesem Fall das vortrainiertes Modell "resnet" auf ImageNet)
Pretraining = Encoder bereits auf einem großen Datensatz (z. B. ImageNet) trainiert > kann nützliche feature erkennen

Warum Datenvorverarbeitung nötig ist:
- Während des Trainings der vortrainierten Gewichte (z. B. auf ImageNet) wurde eine spezifische Vorverarbeitung auf die Bilder angewendet, z. B. Normalisierung, Größenanpassung oder Farbraumkonvertierung.
- Um die bestmögliche Leistung mit den vortrainierten Gewichten zu erzielen, sollten die Eingabedaten genauso vorverarbeitet werden, wie es während des Pretrainings gemacht wurde.

Was `get_preprocessing_fn` macht:
Diese Funktion liefert eine Vorverarbeitungsfunktion (preprocess_input), die genau die gleichen Transformationen durchführt, die während des Pretrainings auf den ImageNet-Datensatz angewendet wurden.





In [ ]:
from segmentation_models_pytorch.encoders import get_preprocessing_fn

preprocess_input = get_preprocessing_fn('resnet18', pretrained='imagenet')

In [ ]:
preprocessed_patches = preprocess_input(patches)

## 4. Write training and  testing loop

### 4.1 Choose optimizer and loss function

In [ ]:
loss = smp.losses.DiceLoss(mode="multiclass")
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)